# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,948.999050,0.950022,1234,949.949072,948.049029,1899.898144,1896.098057
6,899.329406,0.103051,1234,899.432458,899.226355,1798.864915,1798.452711
7,153.795184,0.167096,1234,153.962280,153.628088,307.924559,307.256176
8,788.218914,0.199443,1234,788.418356,788.019471,1576.836713,1576.038942
9,347.335444,0.798397,1234,348.133841,346.537047,696.267681,693.074093
10,160.172917,-0.309020,1234,159.863897,160.481937,319.727794,320.963874
11,691.899018,0.779417,1234,692.678436,691.119601,1385.356871,1382.239202
12,855.995552,0.733681,1234,856.729233,855.261870,1713.458466,1710.523741
13,106.027093,-0.658511,1234,105.368582,106.685605,210.737163,213.371209
14,898.903655,0.618694,1234,899.522350,898.284961,1799.044699,1796.569922


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-949.949072,948.049029
6,-899.432458,899.226355
7,-153.962280,153.628088
8,-788.418356,788.019471
9,-348.133841,346.537047
10,-159.863897,160.481937
11,-692.678436,691.119601
12,-856.729233,855.261870
13,-105.368582,106.685605
14,-899.522350,898.284961


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-949.949072,948.049029
6,-899.432458,899.226355
7,-153.962280,153.628088
8,-788.418356,788.019471
9,-348.133841,346.537047
10,-159.863897,160.481937
11,-692.678436,691.119601
12,-856.729233,855.261870
13,-105.368582,106.685605
14,-899.522350,898.284961


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-949.94907217,  948.04902863],
       [-899.43245752,  899.22635542],
       [-153.96227963,  153.62808822],
       [-788.41835633,  788.01947124],
       [-348.13384072,  346.53704669],
       [-159.86389709,  160.48193705],
       [-692.67843558,  691.11960096],
       [-856.72923292,  855.26187037],
       [-105.36858172,  106.6856047 ],
       [-899.52234972,  898.28496125]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1897.898144,-1896.098057
6,1796.864915,-1798.452711
7,305.924559,-307.256176
8,1574.836713,-1576.038942
9,694.267681,-693.074093
10,317.727794,-320.963874
11,1383.356871,-1382.239202
12,1711.458466,-1710.523741
13,208.737163,-213.371209
14,1797.044699,-1796.569922


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
